# COMS W1002 Computing in Context: Computational Finance  
## Professor Karl Sigman
## Option Pricing Using Monte Carlo Simulation

__Problem 1:__ For the values of $p=0.5$ and $p=0.75$ obtain an estimate of $p$ by using Monte Carlo Simulation with $n=100, n=1000, n=10,000$ You will generate $n$ _iid_ Bernoulli $(p)$ random variables $B_1, ..., B_n$ and use as the estimate $$ \hat{p}= \frac{1}{n}\sum_{i=1}^{n}B_i $$ 
This is to show you (convince you) how accurate the Monte Carlo method can be, and how the Strong Law of Large Numbers (SLLN) works in practice.

In [333]:
# write your code here
# you may add more cells as needed

#import random number generator
import random
#number of trials
N = 10000
#probability of success
prob = .5

def bernoulli(p):
    u = random.random()
    if u<p:
        return 1
    return 0

sum([bernoulli(prob)== 1 for i in range(N)])/N

0.5076

Output if p = .5:

N = 100, p = .46

N = 1000, p = .507

N = 10000, p = .5054

In [39]:
#number of trials
N = 100
#probability of success
p = .75

sum([bernoulli()== 1 for i in range(N)])/N        

0.74

Output if p = .75:

N = 100, p = .88

N = 1000, p = .748

N = 10000, p = .7454

__Problem 2:__ Recall the formula for computing the price $C_0$ of an option (derivative of the BLM stock prices) That yields a payoff at time $T$, denote by $C_T$:
$$ C_0 = \frac{1}{(1+r)^T}E^*(C_T), $$  

where $*$ refers to the fact that we must use the value $p^*$ instead of the original (real) $P$ for the up/down probability of the BLM. (The real value of $P$ is not needed for priciing.) Also recall that for $C_T = (S_T - K)^+$, the European call option, the expected value, $E^*(S_T-K)^+$ can be computed explicitly yielding the famous Black-Scholes-Merton option pricing formula:  
$$ C_0 = \frac{1}{(1+r)^T}\sum_{k=0}^{T} \left( \begin{array}{c}
T \\ k  \end{array} \right )(p^*)^k(1-p^*)^{T-k}(u^k d^{T-k}S_0-K)^+ . $$  

You are to use this formula to exactly obtain the price on the one hand, and then use Monte Carlo simulation on the other hand to compare and thus see how accurate the Monte Carlo method can be. 

Here are the parameters to use: $T = 10, r = 0.05, u = 1.15, d= 1.01, p =.05, S_0 = 50, K = 70$. Recall that 
$$ p^* = \frac{1+r-d}{u-d} . $$  

For the Monte Carlo, use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging) to see how it gets more accurate as $n$ increases.

In [262]:
#C0 using implicit formula for the european call option

#Tools for combinations
import math 

def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

#Time increments
T = 10
#interest rate
r = .05
#Up value
u = 1.15
#Down value
d = 1.01
#probability
p = .05
#Starting price
S0 = 50
#Strike Place
K = 70
#Current day increment
i = 0
#expected probability
pstar = (1 + r - d) / (u - d)
#longprice
lp = 0
lpsum = 0
#Option price
C0 = 0
#Track up or down value
up = 0
#Combination of T and i
C = 0
#total sum for series in sigma
totalsum = []

#Black-Scholes-Merton Formula
while i <= 10:
    up = ((u**i)*(d**(T - i))*S0-K)
    if up <= 0:
        up = 0
    lp = (pstar**i)*((1 - pstar)**(T-i))* up
    C = nCr(T,i)
    lpsum = lp*C
    totalsum.append(lpsum)
    i = i + 1

totalsum = sum(totalsum)
C0 = totalsum * (1/((1 + r)**T))

print(C0)

7.943399485843426


In [499]:
#Sum of cost at BLM
blmsum = 0
#number of trials
i = 10000
#Variable for recursion
r = 0
#Total cost of BLM
blmtotal = 0

#Generate Y variable
def Y(p,u,d):
    if bernoulli(p):
        return u
    else:
        return d

#Generate BLM
def BLM(S0,n,p,u,d):
    BLMpath = [0 for k in range(n+1)]
    BLMpath[0] = S0
    Ysamples = [Y(p,u,d) for k in range (n)]    
    S = S0
    for k in range(1,n+1):
        S *= Ysamples[k-1]
        BLMpath[k] = S
    return BLMpath

while r <= i:
    blmsum = BLM(50, 10, 4/14, 1.15, 1.01)
    blmsum = blmsum[-1]
    r = r + 1
blmtotal = blmsum/i*1000
print(blmtotal)

8.15291926817874


n(100) = CT = 8.15
n(1000) = CT = 7.16
n(10000) = CT = 8.15

__Problem 3__: With the same parameters as in Problem 2, price the _Asian call option:_ The payoff $C_T$ at atime $T$ is based on the average value of the stock over the $T$ time units:
$$ C_T = \left( \frac{1}{T} \sum_{i=1}^{T} S_i-K \right)^+ $$  

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In [461]:
#Sum of cost at BLM
blmsum = 0
#number of trials
i = 1000
#recursion
r = 0
#Total cost of BLM
blmtotal = 0

#Generate Y variable
def Y(p,u,d):
    if bernoulli(p):
        return u
    else:
        return d

#Generate BLM
def BLM(S0,n,p,u,d):
    BLMpath = [0 for k in range(n+1)]
    BLMpath[0] = S0
    Ysamples = [Y(p,u,d) for k in range (n)]    
    S = S0
    for k in range(1,n+1):
        S *= Ysamples[k-1]
        BLMpath[k] = S 
    return BLMpath

blmtotal = 0
pathtotal = 0
#Recursion to find the sum of path values
while r <= i:
    blmsum = 0
    blmsum = BLM(50, 10, 4/14, 1.15, 1.01)
    blmsum = sum(blmsum)
    pathtotal = blmsum + pathtotal
    r = r + 1
pathtotal = pathtotal / 10
CT = pathtotal/(i*10)
print(CT)

7.081728848900543


n(100) = CT = 7.22
n(1000) = CT = 7.11
n(10000) = CT = 7.106


__Problem 4__: With the same parameters as in Problem 2, but the additional parameters $n_1 = 4, n_2 =6,$ and $b=66$: price a _down and out barrier option_, that has payoff at time $T$ of  

$$ C_T = (S_T -K)^+ I \{ S_{n_1} \geq b, S_{n_2} \geq b \}.$$ 

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In the above, recall that for any event $A$, $I\{A\}$ denotes the _indicator_ random variable defined by  
$$ I\{A\} = \left\{ \begin{array}{ll} 1 & \mbox{if $A$ occurs,} \\ 0 & \mbox{if $A$ does not occur.} \end{array} \right. $$

  
Here, $A = \{S_{n_1} \geq b, S_{n_2} \geq b\}$.


In [504]:
#Unable to finish
def indicator(p):
    u = random.random()
    if u<p:
        return 1
    return 0

indicator(4/14)


#Generate BLM
def BLM(S0,n,p,u,d):
    BLMpath = [0 for k in range(n+1)]
    BLMpath[0] = S0
    Ysamples = [Y(p,u,d) for k in range (n)]    
    S = S0
    for k in range(1,n+1):
        S *= Ysamples[k-1]
        BLMpath[k] = S
    return BLMpath


blmsum = BLM(50, 10, 4/14, 1.15, 1.01)

for i in len(blmsum):
    if blmsum[i]




8.15291926817874
